In [1]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
from tqdm import tqdm
import re

In [2]:
def connMySQL():
    conn = pymysql.connect('localhost', 'root', '123jimmyD', 'makeup')
    conn.autocommit(1)
    cur = conn.cursor()
    return cur

In [3]:
def remove_special_char(text):
    try:
        text = re.sub(r'[^a-zA-Z0-9\u4e00-\u9fff]' , '',text)
    except:
        pass
    return text

# Blogger List

In [4]:
df = pd.read_csv('finalized_2788bloggers.csv')

In [5]:
#Create table
connMySQL().execute('''
                        create table bloggers 
                        (id int(10) not null, 
                        blogger_name varchar(50), 
                        blogger_id varchar(11), 
                        blogger_gender varchar(6), 
                        blogger_region varchar(6), 
                        blogger_tag varchar(100), 
                        blogger_follows int(6), 
                        blogger_fans int(9), 
                        blogger_posts int(7), 
                        primary key (id))
                        ''')

C:\Users\panji\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1681, 'Integer display width is deprecated and will be removed in a future release.')
  result = self._query(query)


0

In [6]:
#store data
for i in df.index:
    row = df.iloc[i]
    connMySQL().execute('insert into bloggers (id,blogger_name,blogger_id, blogger_gender, blogger_region,blogger_tag,blogger_follows,blogger_fans, blogger_posts) values (%s,%s,%s,%s,%s,%s,%s,%s,%s)',
                        (int(row[0]),row[1],str(row[2]),row[3], remove_special_char(str(row[4])),remove_special_char(str(row[5])), int(row[6]),int(row[7]),int(row[8])))

# Post List

In [7]:
#Create table
posts = '''
                  CREATE TABLE posts
                  (post_id VARCHAR(20) NOT NULL,
                   blogger_id bigint(15) NOT NULL,
                   post VARCHAR(10000) DEFAULT NULL,
                   picture VARCHAR(1200) DEFAULT NULL,
                   video VARCHAR(1000) DEFAULT NULL,
                   date VARCHAR(10) DEFAULT NULL,
                   no_likes INT(10) DEFAULT NULL,
                   no_comments INT(10) DEFAULT NULL,
                   no_reposts INT(10) DEFAULT NULL,
                   no_topics VARCHAR(500) DEFAULT NULL,
                   no_related_accounts VARCHAR(1250) DEFAULT NULL,
                   PRIMARY KEY (post_id))
              '''
connMySQL().execute(posts)

0

In [ ]:
#Store data
try:   
    cur = connMySQL()
    for i in tqdm(range(0,14)):
        lstpath = 'uid_list/uid_list'+str(i)+'.csv'
        uid_list = pd.read_csv(lstpath, header = None)[0].tolist()
        for uid in uid_list:
            inpath = 'post/' + str(i) + '/' + str(uid) + '/'+ str(uid) + '.csv'
            try:
                posts = pd.read_csv(inpath)
            except Exception as e :
                print(e)
                continue
            posts = posts.astype(object).where(pd.notnull(posts), None)
            for k in range(len(posts)):
                post =  remove_special_char(posts['正文'][k])
                no_topics = remove_special_char(posts['话题'][k])
                related_accounts = remove_special_char(posts['@用户'][k])
                try:
                    cur.execute('INSERT INTO posts (post_id,blogger_id,post,picture,video,date,no_likes,no_comments,no_reposts,no_topics,no_related_accounts) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)',
                                (posts['bid'][k], str(uid), post, posts['原始图片url'][k], posts['视频url'][k], posts['日期'][k], int(posts['点赞数'][k]), int(posts['评论数'][k]),int(posts['转发数'][k]),no_topics,related_accounts))
                except Exception as e:
                    if 'Duplicate' not in str(e):
                        print(e,i,uid,k)
                    continue
except Exception as e:
    print(e,i,uid,k)

# Old Brand List

In [10]:
df = pd.read_csv('filtered_brands_df.csv',header = None,names = ['name', 'indx'])

In [11]:
dic = dict()
for i in range(3380):
    try:
        lst = list()
        for j in df[df.indx==i]['name']:
            lst.append(j)
        dic[i] = {'name':lst}
    except:
        continue

In [12]:
brand = pd.DataFrame(dic).T
brand['index'] = brand.index

In [13]:
#Create table
connMySQL().execute('''
                        create table old_brands 
                        (brand_id int(5) not null, 
                        name varchar(100),
                        primary key (brand_id))
                        ''')

0

In [14]:
#store data
for i in brand.index:
    row = brand.iloc[i]
    connMySQL().execute('insert into old_brands (brand_id,name) values (%s,%s)',
                        (int(row[1]),str(row[0])))

# Product not cleaned List

In [15]:
connMySQL().execute('''
                        create table products_notcleaned 
                        (blogger_id varchar(20) DEFAULT NULL, 
                        post_id varchar(10), 
                        post varchar(10000) DEFAULT NULL, 
                        picture varchar(1200) DEFAULT NULL, 
                        video varchar(1000) DEFAULT NULL, 
                        position varchar(500) DEFAULT NULL, 
                        date varchar(10) DEFAULT NULL, 
                        tool varchar(100) DEFAULT NULL, 
                        no_likes int(10) DEFAULT NULL, 
                        no_comments int(10) DEFAULT NULL, 
                        no_reposts int(10) DEFAULT NULL, 
                        no_topics varchar(1000) DEFAULT NULL,
                        no_related_accounts varchar(1250) DEFAULT NULL, 
                        product varchar(500), 
                        brand_id int(5) DEFAULT NULL, 
                        primary key (post_id))
                        ''')

0

In [ ]:
import os
for i in tqdm(range(14)):#0-14
    path = "product_name/"+str(i) #文件夹目录
    files= os.listdir(path) #得到文件夹下的所有文件名称
    for file in files: #遍历文件夹
        if file.startswith('._'):
            continue
        id = file.split('_')[0]
        df = pd.read_csv(path+"/"+file) #打开文件
              
        for i in df.index:
            row = df.iloc[i]
            
            try:
                connMySQL().execute('INSERT INTO products_notcleaned (blogger_id, post_id, post, picture,video, position,date, tool,no_likes, no_comments, no_reposts, no_topics, no_related_accounts,product, brand_id) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)',
                                (id,remove_special_char(str(row[1])), row[2],str(row[3]),str(row[4]),str(row[5]), row[6], str(row[7]),int(row[8]), int(row[9]), int(row[10]),str(row[11]), str(row[12]),remove_special_char(str(row[13])),str(row[14])))
            except Exception as e:
                if 'Duplicate' not in str(e):
                    print(id,row,e)
                continue

# Product List

In [16]:
connMySQL().execute( '''
                    CREATE TABLE products
                    (prod_url VARCHAR(500) NOT NULL,
                     post_id VARCHAR(20) NOT NULL,
                     prod_name VARCHAR(50) NOT NULL,
                     brand_name VARCHAR(50) NOT NULL,
                     price INT(10) DEFAULT NULL, 
                     popularity INT(10) DEFAULT NULL,
                     score FLOAT DEFAULT NULL,
                     pos_tag VARCHAR(100) DEFAULT NULL,
                     neg_tag VARCHAR(100) DEFAULT NULL)
    
                 ''')

0

In [ ]:
cur = connMySQL()
for i in tqdm(range(0,14)):
    path = 'cleaned_product_info/result_uid'+str(i)+'.csv'
    product = pd.read_csv(path)
    product = product.astype(object).where(pd.notnull(product), None)
    for k in range(len(product)):
        try:
            product_price = product['price'][k]
            if '-' in str(product_price):
                prices_lst = product_price.split('-')
                product_price = (float(prices_lst[0])+float(prices_lst[1]))/2
            cur.execute('INSERT INTO products (prod_url,post_id,prod_name,brand_name,price,popularity,score,pos_tag,neg_tag)VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)', (product['url'][k], product['bid'][k], product['name'][k], product['brand'][k], int(float(product_price)), int(float(product['popularity'][k])), float(product['score'][k]), str(product['posi'][k]), str(product['neg'][k])))
        except Exception as e:
            if 'Duplicate' not in str(e):
                print(e,i,k)
            continue

# New Brand List

In [38]:
brand_data = pd.read_csv('brand_data_final.csv')
brand_data = brand_data.astype(object).where(pd.notnull(brand_data), None)

In [20]:
connMySQL().execute( '''
                    CREATE TABLE brands
                  (brand_id INT(10) NOT NULL, 
                   brand_name VARCHAR(50) NOT NULL,
                   brand_gender VARCHAR(10) DEFAULT NULL, 
                   brand_region VARCHAR(5) DEFAULT NULL, 
                   brand_tag VARCHAR(100) DEFAULT NULL, 
                   brand_follows VARCHAR(20) DEFAULT NULL, 
                   brand_fans VARCHAR(20) DEFAULT NULL, 
                   brand_posts VARCHAR(20) DEFAULT NULL,
                   brand_baidu_idx INT(10) DEFAULT NULL,
                   PRIMARY KEY (brand_id))

                 ''')

0

In [39]:
brand_data.head()
brand_data.drop

,Unnamed: 0,name,uid,gender,region,tag,follows,fans,posts,brandsCN,brandsEN,brand,hot_topic
0,0,井田志Gridnote,5619422243,male,北京,原本一生（北京）商业有限公司,363,3487,494,井田,CANMAKE,canmake/井田,1
1,1,宝拉珍选PaulasChoice,2040921197,female,上海,美国宝拉珍选化妆品官方微博,271,260000,7286,宝拉珍选,PAULA‘S CHOICE,Paula‘s Choice/宝拉珍选,0
2,2,理肤泉larocheposay,2051652844,female,海外,"理肤泉larocheposay, 官方微博",68,200000,6883,理肤泉,LAROCHE POSAY,Laroche Posay/理肤泉,1
3,3,红门雅顿,1791154147,female,海外,伊丽莎白雅顿官方微博,268,250000,10000,雅顿,ELIZABETH ARDEN,Elizabeth Arden/雅顿,0
4,4,Curel珂润,2460538310,female,海外,花王Curel珂润官方微博,149,60000,6928,珂润,CUREL,Curel/珂润,0


In [ ]:
try:   
    cur = connMySQL()
    for i in tqdm(range(len(brand_data))):
        try:
            cur.execute('INSERT INTO brands (brand_id,brand_name,brand_gender,brand_region,brand_tag,brand_follows,brand_fans,brand_posts,brandsCN,brandsEN,brand,hot_topic) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)',
                        (int(brand_data['uid'][i]), brand_data['name'][i], brand_data['gender'][i], brand_data['region'][i], brand_data['tag'][i], int(brand_data['follows'][i]), int(brand_data['fans'][i]), int(brand_data['posts'][i]),brand_data['brandsCN'][i],brand_data['brandsEN'][i],brand_data['brand'][i],int(brand_data['hot_topic'][i])))
        except Exception as e:
            if 'Duplicate' not in str(e):
                print(e,i)
            continue
except Exception as e:
    print(e)